In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,explained_variance_score,r2_score
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import layers
import math
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [2]:
data = pd.read_csv("./dataset/train.csv")

In [3]:
data.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6368 entries, 0 to 6367
Data columns (total 15 columns):
Product_id          6368 non-null object
Stall_no            6363 non-null float64
instock_date        6368 non-null object
Market_Category     6368 non-null int64
Customer_name       6157 non-null object
Loyalty_customer    6368 non-null object
Product_Category    6368 non-null object
Grade               6368 non-null int64
Demand              6368 non-null int64
Discount_avail      6330 non-null float64
charges_1           6170 non-null float64
charges_2 (%)       6163 non-null float64
Minimum_price       6330 non-null float64
Maximum_price       6025 non-null float64
Selling_Price       6327 non-null float64
dtypes: float64(7), int64(3), object(5)
memory usage: 746.3+ KB


In [5]:
price_prediction_feature= data.copy()

In [6]:
price_prediction_feature.isna().sum()

Product_id            0
Stall_no              5
instock_date          0
Market_Category       0
Customer_name       211
Loyalty_customer      0
Product_Category      0
Grade                 0
Demand                0
Discount_avail       38
charges_1           198
charges_2 (%)       205
Minimum_price        38
Maximum_price       343
Selling_Price        41
dtype: int64

In [7]:
# labels = price_prediction_feature.pop("Selling_Price")

In [8]:
range_val = price_prediction_feature["Maximum_price"]-price_prediction_feature["Minimum_price"]

In [9]:
range_val_mean = range_val.mean()

In [10]:
range_val_mean

2901.928726422968

In [11]:
minimum_mean_price = []
maximum_mean_price = []
for i,j in zip(price_prediction_feature["Minimum_price"],price_prediction_feature["Maximum_price"]):
    if i > 0:
        minimum_mean_price.append(i)
    else:
        minimum_mean_price.append(j-range_val_mean)
    if j > 0:
        maximum_mean_price.append(j)
    else:
        maximum_mean_price.append(i+range_val_mean)
len(minimum_mean_price),type(minimum_mean_price)

(6368, list)

In [12]:
labels = pd.DataFrame(minimum_mean_price)
labels.head()

,0
0,2983.0
1,7495.0
2,5752.0
3,5090.0
4,2430.0


In [13]:
labels[1] = maximum_mean_price
labels.head()

,0,1
0,2983.0,4713.0
1,7495.0,10352.0
2,5752.0,7309.0
3,5090.0,20814.0
4,2430.0,9261.0


In [14]:
# price_prediction_feature["Selling_Price"] = labels
# type(price_prediction_feature),price_prediction_feature.shape

In [15]:
labels = labels.fillna(labels.mean())

In [16]:
labels.isna().sum()

0    0
1    0
dtype: int64

In [17]:
labels["Selling_Price"] = price_prediction_feature.pop("Selling_Price")
labels.head()

,0,1,Selling_Price
0,2983.0,4713.0,4185.947700
1,7495.0,10352.0,9271.490256
2,5752.0,7309.0,6785.701362
3,5090.0,20814.0,13028.917824
4,2430.0,9261.0,906.553935


In [18]:
modified = []
for i,j,k in zip(labels[0],labels[1],labels["Selling_Price"]):
    if k > 0:
        modified.append([(i+j)/2,k])
    else:
        modified.append([(i+j)/2,(i+j)/2])

In [19]:
modified = np.array(modified)
_ = price_prediction_feature.pop("Maximum_price")
_ = price_prediction_feature.pop("Minimum_price")
modified.shape,price_prediction_feature.shape

((6368, 2), (6368, 12))

In [20]:
price_prediction_feature = price_prediction_feature.fillna(price_prediction_feature.mean())

In [21]:
price_prediction_feature.isna().sum()

Product_id            0
Stall_no              0
instock_date          0
Market_Category       0
Customer_name       211
Loyalty_customer      0
Product_Category      0
Grade                 0
Demand                0
Discount_avail        0
charges_1             0
charges_2 (%)         0
dtype: int64

In [22]:
# labels["mean"] = (labels[0]+labels[1])/2
# labels.head()

In [23]:
# _ = labels.pop(0)
# _ = labels.pop(1)

In [24]:
labels = pd.concat([price_prediction_feature,pd.DataFrame(modified)],axis=1)
labels.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),0,1
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,3848.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,8923.5,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,6530.5,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,12952.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,5845.5,906.553935


In [25]:
labels.isna().sum()

Product_id            0
Stall_no              0
instock_date          0
Market_Category       0
Customer_name       211
Loyalty_customer      0
Product_Category      0
Grade                 0
Demand                0
Discount_avail        0
charges_1             0
charges_2 (%)         0
0                     0
1                     0
dtype: int64

In [26]:
price_prediction_feature = labels.copy()

In [27]:
# price_prediction_feature = price_prediction_feature.replace(np.nan," ")
_ = price_prediction_feature.pop("Customer_name")
product_cat = price_prediction_feature.pop("Product_Category")
Grade = price_prediction_feature.pop("Grade")

In [28]:
price_prediction_feature.shape

(6368, 11)

In [29]:
price_prediction_feature.isna().sum()

Product_id          0
Stall_no            0
instock_date        0
Market_Category     0
Loyalty_customer    0
Demand              0
Discount_avail      0
charges_1           0
charges_2 (%)       0
0                   0
1                   0
dtype: int64

In [30]:
# maximum = max(price_prediction_feature[0]+price_prediction_feature[1])
# minimum = min(+price_prediction_feature[1]+price_prediction_feature[0])
# maximum,minimum

In [31]:
out = price_prediction_feature.pop(1)

In [32]:
_ = price_prediction_feature.pop("Product_id")

In [33]:
encoder = OrdinalEncoder()

In [34]:
price_prediction_feature = encoder.fit_transform(price_prediction_feature)
price_prediction_feature

array([[3.700e+01, 1.502e+03, 2.000e+00, ..., 2.770e+02, 1.100e+01,
        2.028e+03],
       [3.800e+01, 4.732e+03, 2.100e+01, ..., 2.980e+02, 1.200e+01,
        4.905e+03],
       [8.000e+00, 1.054e+03, 2.260e+02, ..., 1.500e+02, 8.000e+00,
        3.991e+03],
       ...,
       [3.600e+01, 3.135e+03, 2.060e+02, ..., 1.840e+02, 2.000e+00,
        5.091e+03],
       [2.700e+01, 3.461e+03, 2.310e+02, ..., 1.680e+02, 2.000e+00,
        3.098e+03],
       [1.000e+01, 3.817e+03, 4.900e+01, ..., 4.540e+02, 6.000e+00,
        2.784e+03]])

In [35]:
product_cat = encoder.fit_transform(np.expand_dims(product_cat,1))
product_cat

array([[3.],
       [3.],
       [0.],
       ...,
       [8.],
       [8.],
       [9.]])

In [36]:
Grade = encoder.fit_transform(np.expand_dims(Grade,1))
Grade

array([[1.],
       [0.],
       [0.],
       ...,
       [3.],
       [0.],
       [0.]])

In [37]:
product_cat.shape,Grade.shape

((6368, 1), (6368, 1))

In [38]:
onehot = OneHotEncoder(sparse=False)

In [39]:
product_cat = onehot.fit_transform(product_cat)
product_cat.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(6368, 10)

In [40]:
Grade = onehot.fit_transform(Grade)
Grade.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(6368, 4)

In [41]:
# price_prediction_feature = preprocessing.normalize(price_prediction_feature)
# price_prediction_feature

In [42]:
price_prediction_feature = preprocessing.minmax_scale(price_prediction_feature)
price_prediction_feature

array([[0.74      , 0.28080015, 0.00809717, ..., 0.5687885 , 0.64705882,
        0.37444609],
       [0.76      , 0.88465134, 0.08502024, ..., 0.61190965, 0.70588235,
        0.90564993],
       [0.16      , 0.19704618, 0.91497976, ..., 0.30800821, 0.47058824,
        0.73689069],
       ...,
       [0.72      , 0.58609086, 0.8340081 , ..., 0.37782341, 0.11764706,
        0.93999261],
       [0.54      , 0.64703683, 0.93522267, ..., 0.3449692 , 0.11764706,
        0.57200886],
       [0.2       , 0.71359133, 0.19838057, ..., 0.93223819, 0.35294118,
        0.5140325 ]])

In [43]:
final = pd.concat([pd.DataFrame(price_prediction_feature),pd.DataFrame(product_cat),pd.DataFrame(Grade)],axis=1)
final.head()

,0,1,2,3,4,5,6,7,8,0,...,4,5,6,7,8,9,0,1,2,3
0,0.74,0.280800,0.008097,1.0,0.676768,0.0,0.568789,0.647059,0.374446,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.76,0.884651,0.085020,1.0,0.505051,0.0,0.611910,0.705882,0.905650,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.16,0.197046,0.914980,1.0,0.090909,0.0,0.308008,0.470588,0.736891,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.00,0.916059,0.080972,1.0,0.474747,0.0,0.090349,0.764706,0.986337,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.12,0.970275,0.218623,1.0,0.343434,1.0,0.227926,0.176471,0.666913,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [44]:
final = np.array(final)

In [45]:
# out = (out-minimum)/(maximum-minimum)

In [46]:
X_train,X_val,Y_train,Y_val = train_test_split(final,out,test_size=0.1,random_state=0)
print(final.shape,out.shape,X_train.shape,X_val.shape,Y_train.shape,Y_val.shape)

(6368, 23) (6368,) (5731, 23) (637, 23) (5731,) (637,)


In [386]:
used_cars_model = tf.keras.Sequential([
    layers.Dense(96,activation="tanh"),
    layers.Dense(96,activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(36,activation="relu"),
    layers.Dense(1)
])
monitor = ModelCheckpoint("./checkpoint/",verbose=1,save_best_only=True)
# monitor = EarlyStopping(restore_best_weights=True,patience=15,verbose=1)
used_cars_model.compile(loss = tf.losses.MeanSquaredError(),optimizer = tf.optimizers.Adam())

In [431]:
Epoch=50
BS = 192
H = used_cars_model.fit(X_train,Y_train,batch_size=BS,callbacks=[monitor],verbose=0,
                        validation_data=(X_val, Y_val),epochs = Epoch)


Epoch 00001: val_loss did not improve from 128778.62500

Epoch 00002: val_loss did not improve from 128778.62500

Epoch 00003: val_loss did not improve from 128778.62500

Epoch 00004: val_loss did not improve from 128778.62500

Epoch 00005: val_loss did not improve from 128778.62500

Epoch 00006: val_loss did not improve from 128778.62500

Epoch 00007: val_loss did not improve from 128778.62500

Epoch 00008: val_loss did not improve from 128778.62500

Epoch 00009: val_loss did not improve from 128778.62500

Epoch 00010: val_loss did not improve from 128778.62500

Epoch 00011: val_loss did not improve from 128778.62500

Epoch 00012: val_loss did not improve from 128778.62500

Epoch 00013: val_loss did not improve from 128778.62500

Epoch 00014: val_loss did not improve from 128778.62500

Epoch 00015: val_loss did not improve from 128778.62500

Epoch 00016: val_loss did not improve from 128778.62500

Epoch 00017: val_loss did not improve from 128778.62500

Epoch 00018: val_loss did not 

In [423]:
from tensorflow.keras.models import load_model
model = load_model("./checkpoint/")

In [424]:
model.evaluate(X_val,Y_val)

20/20 [==============================] - 0s 1ms/step - loss: 128778.6016


128778.6015625

In [425]:
#Get predicted value from validation data 
Y_predicted =  model.predict(X_val)
print("predicted array",Y_predicted[:10])

predicted array [[6043.784  ]
 [3778.8923 ]
 [2594.7952 ]
 [7309.2866 ]
 [2435.164  ]
 [4768.903  ]
 [ 558.9141 ]
 [ 587.9146 ]
 [5087.2954 ]
 [ 850.84845]]


In [426]:
#Match the dimensions of true and predicted value
print(Y_predicted.shape,Y_val.shape)
Y_predicted = np.reshape(Y_predicted,(Y_val.shape))
print(Y_predicted.shape,Y_val.shape)

(637, 1) (637,)
(637,) (637,)


In [427]:
print("Prediction Summary scaled")
print("Mean Squared Error",mean_squared_error(Y_val,Y_predicted))
print("RMSE",sqrt(mean_squared_error(Y_val,Y_predicted)))
print("R2 score =", round(r2_score(Y_val, Y_predicted), 2))
print("R2 score near to 1 means model is well fitted")
print("Explain variance score =", round(explained_variance_score(Y_val, Y_predicted), 4))

Prediction Summary scaled
Mean Squared Error 128778.60437537344
RMSE 358.85735937190066
R2 score = 0.99
R2 score near to 1 means model is well fitted
Explain variance score = 0.9859


In [428]:
# checking for some real and predicted values
for i,j in zip(Y_val[:20],Y_predicted[:20]):
    print("True Value",round(i,2),"       Predicted Value",round(j,2))

True Value 6083.66        Predicted Value 6043.78
True Value 3836.78        Predicted Value 3778.89
True Value 2594.3        Predicted Value 2594.8
True Value 7183.23        Predicted Value 7309.29
True Value 2552.5        Predicted Value 2435.16
True Value 4856.32        Predicted Value 4768.9
True Value 485.78        Predicted Value 558.91
True Value 530.83        Predicted Value 587.91
True Value 5225.79        Predicted Value 5087.3
True Value 965.22        Predicted Value 850.85
True Value 4316.13        Predicted Value 4337.21
True Value 4021.92        Predicted Value 4065.37
True Value 5571.69        Predicted Value 5433.66
True Value 3941.76        Predicted Value 3945.91
True Value 7180.59        Predicted Value 7105.05
True Value 12601.47        Predicted Value 13855.05
True Value 12406.1        Predicted Value 12147.08
True Value 2921.96        Predicted Value 2853.68
True Value 788.17        Predicted Value 861.54
True Value 5465.2        Predicted Value 5338.53


In [429]:
model.save("Best_model",save_format="h5")